<h1>Phish Song Predictor</h1>

In [1]:
from bs4 import BeautifulSoup
import string
import os
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#Set up class called setlist that a two attributes, date and songlist
class Setlist:
    def __init__(self):
        self.date = ""
        self.songlist = [] 

<h3>Beautiful Soup Scrape</h3>

Use Beautiful Soup to scrape phish.net to pull out setlists & date from html

In [18]:
dateDict = {}
all_setlists = []

#years we want to scrape
yearRange = range(1986,2016)


for year in yearRange:
    soup = BeautifulSoup(open('data/{0}.html'.format(str(year)),encoding = "ISO-8859-1"),"lxml")
    dateDivs = soup.find_all('div', attrs={'class': 'setlist'})
    for div in dateDivs:
        slist = Setlist()
        link = div.h2.a
        if link.has_attr('href'):
            slist.date = str(link['href']).split("=")[1]
        song_groups_soup = BeautifulSoup(str(div), "lxml")
        song_groups = song_groups_soup.find_all('p')
        for group in song_groups:
            songs_soup = BeautifulSoup(str(group), "lxml")
            song_links = songs_soup.find_all('a')
            for song_link in song_links:
                if "?teaseid=" not in str(song_link):
                    song_name = song_link['href'].replace("http://phish.net/song/", "").replace("http://phish.net/faq/", "").replace("http://www.livephish.com/", "")
                    if song_name not in slist.songlist:
                        slist.songlist.append(song_name)
    
        all_setlists.append(slist)
        


<h3>Create ordered songlist</h3>

In [19]:
setlists = [i.songlist for i in all_setlists]
dates = [i.date for i in all_setlists]

In [20]:
# song_list = open('song_list.txt', 'w')
# for item in setlists:
#   song_list.write("%s" % item)

In [32]:
#read songs for text file
f = open('song_list.txt', 'r')
songs_read = f.read()

<h3>Create Pandas df to explore data</h3>

In [22]:
phish1 = pd.DataFrame(setlists)
phish2 = pd.DataFrame(dates)

phish = phish2.join(phish1,  lsuffix='_caller', how="left")
columns = list(range(1, 51))
phish.columns = ["dates"]+columns

In [23]:
# df = pd.DataFrame(setlists)

# phish3 = df.transpose()
# phish3.columns = dates

# phish3[:10]
#phish.to_csv("phish.csv")

In [24]:
phish_csv = pd.read_csv('phish.csv')
del phish_csv['Unnamed: 0']
phish_csv = phish_csv.transpose()
phish_csv.columns = phish_csv.iloc[0]
phish3 = phish_csv.drop(['dates'])
phish3 [:10]

dates,1986-02-03,1986-02-28,1986-04-01,1986-04-15,1986-04-29,1986-05-16,1986-05-17,1986-05-24,1986-06-01,1986-09-03,...,2015-08-16,2015-08-20,2015-08-21,2015-08-22,2015-08-23,2015-09-04,2015-09-05,2015-09-06,2015-12-30,2015-12-31
1,slave-to-the-traffic-light,NaN,quinn-the-eskimo,acdc-bag,NaN,NaN,halleys-comet,dog-log,NaN,NaN,...,golgi-apparatus,the-bunny,simple,divided-sky,punch-you-in-the-eye,tube,no-men-in-no-mans-land,the-landlady,sample-in-a-jar,the-moma-dance
2,mikes-song,NaN,have-mercy,dear-mrs-reagan,NaN,NaN,back-porch-boogie-blues,halleys-comet,NaN,NaN,...,undermind,NaN,the-dogs,the-moma-dance,buffalo-bill,ghost,martian-monster,free,free,possum
3,daves-energy-guide,NaN,harry-hood,prep-school-hippie,NaN,NaN,NaN,back-porch-boogie-blues,NaN,NaN,...,julius,NaN,the-man-who-stepped-into-yesterday,mound,a-song-i-heard-the-ocean-sing,halleys-comet,nicu,the-moma-dance,simple,wolfmans-brother
4,you-enjoy-myself,NaN,the-pendulum,quinn-the-eskimo,NaN,NaN,NaN,NaN,NaN,NaN,...,555,NaN,avenu-malkenu,army-of-one,limb-by-limb,undermind,stealing-time-from-the-faulty-plan,seven-below,back-on-the-train,birds-of-a-feather
5,alumni-blues,NaN,daves-energy-guide,slave-to-the-traffic-light,NaN,NaN,NaN,NaN,NaN,NaN,...,nothing,NaN,free,scabbard,waiting-all-night,yarmouth-road,bouncing-around-the-room,prince-caspian,waiting-all-night,i-didnt-know
6,letter-to-jimmy-page,NaN,icculus,makisupa-policeman,NaN,NaN,NaN,NaN,NaN,NaN,...,no-men-in-no-mans-land,NaN,happy-birthday-to-you,sample-in-a-jar,theme-from-the-bottom,bathtub-gin,555,backwards-down-the-number-line,555,happy-birthday-to-you
7,prep-school-hippie,NaN,you-enjoy-myself,have-mercy,NaN,NaN,NaN,NaN,NaN,NaN,...,stash,NaN,the-wedge,tube,maze,waiting-all-night,winterqueen,the-line,roggae,martian-monster
8,run-like-an-antelope,NaN,help-on-the-way,all-blues,NaN,NaN,NaN,NaN,NaN,NaN,...,david-bowie,NaN,mock-song,halfway-to-the-moon,the-line,horn,split-open-and-melt,scent-of-a-mule,the-dogs,reba
9,tonight,NaN,slipknot,dog-log,NaN,NaN,NaN,NaN,NaN,NaN,...,possum,NaN,roggae,camel-walk,stash,the-wedge,limb-by-limb,saw-it-again,46-days,walls-of-the-cave
10,the-pendulum,NaN,acdc-bag,possum,NaN,NaN,NaN,NaN,NaN,NaN,...,down-with-disease,NaN,rift,how-many-people-are-you,reba,46-days,roggae,halfway-to-the-moon,chalk-dust-torture,the-wedge


**Set Up N-Grams lists**

In [25]:
songs_ordered = []

for setlist in setlists:
    for song in setlist:
        songs_ordered.append(song)

three_grams = []
for i in range(0,len(songs_ordered)-3):
    three_grams.append([songs_ordered[i],songs_ordered[i+1],songs_ordered[i+2]])
    
two_grams = []
for i in range(0,len(songs_ordered)-2):
    two_grams.append([songs_ordered[i],songs_ordered[i+1]])

print('Three Grams:')
print(three_grams[:5],'\n')
print('Two Grams:')
print(two_grams[:5])

Three Grams:
[['slave-to-the-traffic-light', 'mikes-song', 'daves-energy-guide'], ['mikes-song', 'daves-energy-guide', 'you-enjoy-myself'], ['daves-energy-guide', 'you-enjoy-myself', 'alumni-blues'], ['you-enjoy-myself', 'alumni-blues', 'letter-to-jimmy-page'], ['alumni-blues', 'letter-to-jimmy-page', 'prep-school-hippie']] 

Two Grams:
[['slave-to-the-traffic-light', 'mikes-song'], ['mikes-song', 'daves-energy-guide'], ['daves-energy-guide', 'you-enjoy-myself'], ['you-enjoy-myself', 'alumni-blues'], ['alumni-blues', 'letter-to-jimmy-page']]


****2-Gram****

In [26]:
twolist = two_grams


# [  
#     ['a','g','c'],
#     ['e','f','g'],
#     ['e','f','e'],
#     ['a','g','c'],
# ]

two = {}
for i,j in twolist:

    if two.get((i,j)) == None:
        
        two[(i,j)] = ((j,1))
    
    else:
        
        two[(i,j)]= ((j,1+int(two[(i,j)][1])))

c = two.items()
two_gram_sort = sorted(c,key=lambda x:x[1][1], reverse=True) 
two_gram_sort[:5]

[(('i-am-hydrogen', 'weekapaug-groove'), ('weekapaug-groove', 318)),
 (('mikes-song', 'i-am-hydrogen'), ('i-am-hydrogen', 304)),
 (('the-horse', 'silent-in-the-morning'), ('silent-in-the-morning', 156)),
 (('the-oh-kee-pa-ceremony', 'suzy-greenberg'), ('suzy-greenberg', 136)),
 (('colonel-forbins-ascent', 'fly-famous-mockingbird'),
  ('fly-famous-mockingbird', 121))]

In [27]:
# i= input("Song 1:" )
# j = input("Song 2:" )
# i="sparkle"
# j="simple"


def two_gram_return(i,j):
    prediction_total = 0
    b = (j)
    for lookup in two_gram_sort:
        if lookup[0][0] == b:
            prediction_total += lookup[1][1]

    print("")
    

    for lookup in two_gram_sort:
        if lookup[0][0] == b:
            print (lookup[1][0]," ",round(float(lookup[1][1]/prediction_total),2)*100,"%")
            

**3-Grams**

In [28]:
import collections
inlist = three_grams


new = {}
for i,j,q in inlist:
    if new.get(((i,j),q)) == None:
        
        new[((i,j),q)] = (q,1)
    
    else:
        
        new[((i,j),q)]= (q,1+int(new[((i,j),q)][1]))
import operator
b = new.items()
three_gram_sort = sorted(b,key=lambda x:x[1][1], reverse=True) 
three_gram_sort[:5]

[((('mikes-song', 'i-am-hydrogen'), 'weekapaug-groove'),
  ('weekapaug-groove', 301)),
 ((('i-am-hydrogen', 'weekapaug-groove'), 'hold-your-head-up'),
  ('hold-your-head-up', 15)),
 ((('tweezer', 'the-horse'), 'silent-in-the-morning'),
  ('silent-in-the-morning', 14)),
 ((('the-lizards', 'mikes-song'), 'i-am-hydrogen'), ('i-am-hydrogen', 13)),
 ((('weekapaug-groove', 'the-horse'), 'silent-in-the-morning'),
  ('silent-in-the-morning', 13))]

In [29]:
def predict(i,j):
    prediction_total = 0


    b = (i,j)
    for lookup in three_gram_sort:
        if lookup[0][0] == b:
            prediction_total += lookup[1][1]
            #print (lookup)
    print("")
    # print("Song 1: %s" % i)
    # print("Song 2: %s \n" % j)
    if prediction_total == 0:
        print('No Three Grams!! \n')
        two_gram_return(i,j)
    else:
        print("Probable Next Song :")
        
    
    

    for lookup in three_gram_sort:
        if lookup[0][0] == b:
            #print("Probable Next Song :")
            print (lookup[1][0]," ",round(float(lookup[1][1]/prediction_total),2)*100,"%")
    


<h2>Final Predictor</h2>


Sample Song Choices: acdc-bag, ya-mar, sparkle, letter-to-jimmy-page, tube, sample-in-a-jar, halleys-comet, ghost

In [31]:
i= input("Song 1:" )
j = input("Song 2:" )
predict(i,j)

Song 1:
Song 2:

No Three Grams!! 


acdc-bag   8.0 %
tube   6.0 %
crowd-control   6.0 %
the-moma-dance   6.0 %
kill-devil-falls   4.0 %
prince-caspian   4.0 %
sample-in-a-jar   4.0 %
stash   4.0 %
wolfmans-brother   4.0 %
llama   2.0 %
the-bunny   2.0 %
soul-shakedown-party   2.0 %
a-song-i-heard-the-ocean-sing   2.0 %
divided-sky   2.0 %
the-line   2.0 %
golgi-apparatus   2.0 %
mikes-song   2.0 %
http://www.secondcity.com/   2.0 %
maze   2.0 %
buried-alive   2.0 %
free   2.0 %
no-men-in-no-mans-land   2.0 %
axilla   2.0 %
https://en.wikipedia.org/wiki/Gene_Ween   2.0 %
grind   2.0 %
gumbo   2.0 %
punch-you-in-the-eye   2.0 %
rift   2.0 %
the-landlady   2.0 %
the-star-spangled-banner   2.0 %
timber-jerry   2.0 %
dinner-and-a-movie   2.0 %
46-days   2.0 %
waiting-all-night   2.0 %
http://www.rollingstone.com/music/pictures/readers-poll-the-10-best-phish-songs-20140702/1-lushington-0013076   2.0 %
fee   2.0 %
the-curtain-with   2.0 %
